## 批量归一化

通常来说，数据标准化预处理对于浅层模型就足够有效了。随着模型训练的进行，当每层中参数更新时，靠近输出层的输出较难出现剧烈变化。但对深层神经网络来说，即使输入数据已做标准化，训练中模型参数的更新依然很容易造成靠近输出层输出的剧烈变化。这种计算数值的不稳定性通常令我们难以训练出有效的深度模型。

批量归一化的提出正是为了应对深度模型训练的挑战。在模型训练时，批量归一化利用小批量上的均值和标准差，不断调整神经网络中间输出，从而使整个神经网络在各层的中间输出的数值更稳定。**批量归一化**和**残差网络**为训练和设计深度模型提供了两类重要思路。

### 1.1 对全连接层批量归一化
使用批量归一化的全连接层的输出为
$$
\phi(BN(x)) = \phi (BN (Wu + b)),
$$
其中$u$为全连接层的输入，$BN$为批量归一化运算符。

对于小批量的**仿射变换的输出**$\mathcal{B} = \{ x^{(1)}, ..., x^{(m)} \}$，其中$x^{(i)} \in \mathbb{R}^d$，则批量归一化的输出为
$$
y^{(i)} = BN(x^{(i)}) \in \mathbb{R}^d.
$$

$BN(\cdot)$的具体步骤如下：

首先对小批量$\mathcal{B}$求均值和方差：
$$
\mu_{\mathcal{B}} \leftarrow \frac{1}{m} \sum_{i=1}^m x^{(i)},\\
\sigma^2_{\mathcal{B}} \leftarrow \frac{1}{m-1} \sum_{i=1}^m (x^{(i)} - \mu_{\mathcal{B}})^2.
$$

其中的平方计算是按元素求平方。接下来，使用按元素开方和按元素除法对$x^{(i)}$标准化：
$$
\hat{x}^{(i)} \leftarrow \frac{x^{(i)} - \mu_{\mathcal{B}}}{\sqrt{\sigma^2_{\mathcal{B}} + \epsilon}},
$$
这里$\epsilon > 0$是一个很小的常数，保证分母大于0。

批量归一化层引入了两个可以学习的模型参数，拉伸（scale）参数$\gamma$和偏移（shift）参数$\beta$。这两个参数和$x^{(i)}$形状相同，皆为$d$维向量。它们与分别做按元素乘法和加法计算：
$$
y^{(i)} \leftarrow \gamma \odot \hat{x}^{(i)} + \beta.
$$

可学习的拉伸和偏移参数保留了不对$x^{(i)}$做批量归一化的可能：
此时只需学出$\gamma = \sqrt{\sigma^2_{\mathcal{B}} + \epsilon}$，$\beta = \mu_{\mathcal{B}}$。我们可以对此这样理解：如果批量归一化无益，理论上，学出的模型可以不使用批量归一化。

### 1.2 对卷积层批量归一化
对卷积层来说，批量归一化发生在卷积计算之后、应用激活函数之前。如果卷积计算输出多个通道，我们需要对这些通道的输出分别做批量归一化，且每个通道都拥有独立的拉伸和偏移参数，并均为**标量**。

设小批量中有$m$个样本。在单个通道上，假设卷积计算输出的高和宽分别为$p$和$q$。我们需要对该通道中$m×p×q$个元素同时做批量归一化。对这些元素做标准化计算时，我们使用相同的均值和方差，即该通道中$m×p×q$个元素的均值和方差。

### 1.3 预测时的批量归一化
使用批量归一化训练时，我们可以将批量大小设得大一点，从而使批量内样本的均值和方差的计算都较为准确。将训练好的模型用于预测时，我们希望模型对于任意输入都有确定的输出。因此，单个样本的输出不应取决于批量归一化所需要的随机小批量中的均值和方差。一种常用的方法是通过移动平均估算整个训练数据集的样本均值和方差，并在预测时使用它们得到确定的输出。可见，和丢弃层一样，批量归一化层在训练模式和预测模式下的计算结果也是不一样的。

### 2.1 实现批量归一化层

In [1]:
import time
import torch
from torch import nn, optim
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def batch_norm(is_training, X, gamma, beta, moving_mean, moving_var, eps, momentum):
    # 判断当前模式是训练模式还是预测模式
    if not is_training:
        # 如果是在预测模式下，直接使用传入的移动平均所得的均值和方差
        X_hat = (X - moving_mean) / torch.sqrt(moving_var + eps)
    else:
        assert len(X.shape) in (2, 4)
        if len(X.shape) == 2:
            # 使用全连接层的情况，计算特征维上的均值和方差
            mean = X.mean(dim=0)
            var = ((X - mean) ** 2).mean(dim=0)
        else:
            # 使用二维卷积层的情况，计算通道维上（axis=1）的均值和方差。这里我们需要保持
            # X的形状以便后面可以做广播运算
            mean = X.mean(dim=0, keepdim=True).mean(dim=2, keepdim=True).mean(dim=3, keepdim=True)
            var = ((X - mean) ** 2).mean(dim=0, keepdim=True).mean(dim=2, keepdim=True).mean(dim=3, keepdim=True)
        # 训练模式下用当前的均值和方差做标准化
        X_hat = (X - mean) / torch.sqrt(var + eps)
        # 更新移动平均的均值和方差
        moving_mean = momentum * moving_mean + (1.0 - momentum) * mean
        moving_var = momentum * moving_var + (1.0 - momentum) * var
    Y = gamma * X_hat + beta  # 拉伸和偏移
    return Y, moving_mean, moving_var

In [2]:
class BatchNorm(nn.Module):
    def __init__(self, num_features, num_dims):
        """
        num_features参数对于全连接层来说应为输出个数，对于卷积层来说则为输出通道数。
        该实例所需指定的num_dims参数对于全连接层和卷积层来说分别为2和4。
        """
        super(BatchNorm, self).__init__()
        if num_dims == 2:
            shape = (1, num_features)
        else:
            shape = (1, num_features, 1, 1)
        # 参与求梯度和迭代的拉伸和偏移参数，分别初始化成0和1
        self.gamma = nn.Parameter(torch.ones(shape))
        self.beta = nn.Parameter(torch.zeros(shape))
        # 不参与求梯度和迭代的变量，全在内存上初始化成0
        self.moving_mean = torch.zeros(shape)
        self.moving_var = torch.zeros(shape)

    def forward(self, X):
        # 如果X不在内存上，将moving_mean和moving_var复制到X所在显存上
        if self.moving_mean.device != X.device:
            self.moving_mean = self.moving_mean.to(X.device)
            self.moving_var = self.moving_var.to(X.device)
        # 保存更新过的moving_mean和moving_var, Module实例的traning属性默认为true, 调用.eval()后设成false
        Y, self.moving_mean, self.moving_var = batch_norm(self.training, 
            X, self.gamma, self.beta, self.moving_mean,
            self.moving_var, eps=1e-5, momentum=0.9)
        return Y

带有批量归一化的LeNet：

In [3]:
import my_utils

net = nn.Sequential(
            nn.Conv2d(1, 6, 5), # in_channels, out_channels, kernel_size
            BatchNorm(6, num_dims=4),
            nn.Sigmoid(),
            nn.MaxPool2d(2, 2), # kernel_size, stride
            nn.Conv2d(6, 16, 5),
            BatchNorm(16, num_dims=4),
            nn.Sigmoid(),
            nn.MaxPool2d(2, 2),
            my_utils.FlattenLayer(),
            nn.Linear(16*4*4, 120),
            BatchNorm(120, num_dims=2),
            nn.Sigmoid(),
            nn.Linear(120, 84),
            BatchNorm(84, num_dims=2),
            nn.Sigmoid(),
            nn.Linear(84, 10)
        )

训练：

In [4]:
batch_size = 256
train_iter, test_iter = my_utils.load_data_fashion_mnist(batch_size=batch_size)

lr, num_epochs = 0.001, 5
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
my_utils.train_cnn(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

training on cpu
epoch 1, loss 0.9759, train acc 0.795, test acc 0.839, time 20.6 sec
epoch 2, loss 0.4477, train acc 0.866, test acc 0.827, time 22.3 sec
epoch 3, loss 0.3624, train acc 0.880, test acc 0.854, time 22.4 sec
epoch 4, loss 0.3277, train acc 0.887, test acc 0.865, time 23.1 sec
epoch 5, loss 0.3040, train acc 0.893, test acc 0.848, time 22.2 sec


查看第一个批量归一化层的学习参数$\gamma$和$\beta$：

In [6]:
net[1].gamma.view((-1,)), net[1].beta.view((-1,))

(tensor([1.2278, 1.1507, 1.0682, 1.0679, 1.1600, 1.1128],
        grad_fn=<ViewBackward>),
 tensor([ 0.4868,  0.0876,  0.1138, -0.5308,  0.3584,  0.0559],
        grad_fn=<ViewBackward>))

Torch的简洁实现：

In [9]:
net = nn.Sequential(
            nn.Conv2d(1, 6, 5), # in_channels, out_channels, kernel_size
            nn.BatchNorm2d(6),
            nn.Sigmoid(),
            nn.MaxPool2d(2, 2), # kernel_size, stride
            nn.Conv2d(6, 16, 5),
            nn.BatchNorm2d(16),
            nn.Sigmoid(),
            nn.MaxPool2d(2, 2),
            my_utils.FlattenLayer(),
            nn.Linear(16*4*4, 120),
            nn.BatchNorm1d(120),
            nn.Sigmoid(),
            nn.Linear(120, 84),
            nn.BatchNorm1d(84),
            nn.Sigmoid(),
            nn.Linear(84, 10)
        )
print(net)

Sequential(
  (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): Sigmoid()
  (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (4): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (5): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): Sigmoid()
  (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (8): FlattenLayer()
  (9): Linear(in_features=256, out_features=120, bias=True)
  (10): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (11): Sigmoid()
  (12): Linear(in_features=120, out_features=84, bias=True)
  (13): BatchNorm1d(84, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (14): Sigmoid()
  (15): Linear(in_features=84, out_features=10, bias=True)
)
